In [ ]:
import pandas as pd
import numpy as np
from numpy import percentile

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
# import plotly.graph_objs as go
from plotly.subplots import make_subplots
# pip install --upgrade pip
# pip install --upgrade Pillow
# pip install wordcloud
from wordcloud import WordCloud
import plotly.express as px
import json

# %pip install missingno
import missingno as msno

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

# pip install pyod
from sklearn.preprocessing import MinMaxScaler
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN

import warnings
warnings.filterwarnings("ignore")

## Load the dataset:

In [ ]:
df = pd.read_csv("Data/Medicare_Part_D_Prescribers_by_Provider_2021.csv")

## Look at  the dataset

In [ ]:
print(df.columns.tolist())

In [ ]:
df_prs = df[['PRSCRBR_NPI','Prscrbr_State_Abrvtn', 'Prscrbr_Type',
             'Tot_Clms','Tot_Drug_Cst', 'Tot_Day_Suply', 'Tot_Benes', 'GE65_Tot_Clms', 
             'Brnd_Tot_Drug_Cst',  'Gnrc_Tot_Drug_Cst', 'Othr_Tot_Drug_Cst',
             'MAPD_Tot_Drug_Cst','PDP_Tot_Drug_Cst','LIS_Drug_Cst','NonLIS_Drug_Cst',
             'Opioid_Tot_Clms','Opioid_Tot_Drug_Cst','Opioid_Tot_Suply', 'Opioid_Tot_Benes', 'Opioid_Prscrbr_Rate', 
             'Opioid_LA_Tot_Clms', 'Opioid_LA_Tot_Drug_Cst', 'Opioid_LA_Tot_Suply', 'Opioid_LA_Tot_Benes', 'Opioid_LA_Prscrbr_Rate', 
             'Antbtc_Tot_Clms','Antbtc_Tot_Drug_Cst', 'Antbtc_Tot_Benes', 
             'Antpsyct_GE65_Tot_Clms', 'Antpsyct_GE65_Tot_Drug_Cst', 'Antpsyct_GE65_Tot_Benes',
             'Bene_Avg_Risk_Scre']]

In [ ]:
df_prs.info()

In [ ]:
df_prs.head(2)

In [ ]:
df_prs['Prscrbr_State_Abrvtn'].nunique()

In [ ]:
df_prs['Prscrbr_State_Abrvtn'].unique()

In [ ]:
df_prs['Prscrbr_Type'].nunique()

## Prescription Claims Overview

In [ ]:
print("Number of Prescribers:", f"{df_prs['PRSCRBR_NPI'].nunique():,}")
print("Number of Day’s Supply for All Claims:", f"{df_prs['Tot_Day_Suply'].sum():,}")
print("Number of Medicare Part D Claims, Including Refills:", f"{df_prs['Tot_Clms'].sum():,}" )
print("Aggregate Cost Paid for All Claims", f"{df_prs['Tot_Drug_Cst'].sum():,}")
print("Number of Medicare Beneficiaries:", f"{df_prs['Tot_Benes'].sum().astype('int'):,}")

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3, 4],
    y=[1.6, 1.6, 1.6, 1.6,1.6],
    mode="text", 
    text=["<span style='font-size:25px'><b>1.29M</b></span>",
                    "<span style='font-size:25px'><b>75.9B</b></span>",
          "<span style='font-size:25px'><b>1.49B</b></span>",
          "<span style='font-size:25px'><b>216B</b></span>",
          "<span style='font-size:25px'><b>186M</b></span>"],
    textposition="bottom center"
))
fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3, 4],
    y=[1.1, 1.1, 1.1, 1.1, 1.1],
    mode="text", 
    text=["Prescribers","Day’s Supply", "Claims", "Drug Cost",  "Beneficiaries"],
    textposition="bottom center"
))
fig.add_hline(y=2.2, line_width=5, line_color='gray')
fig.add_hline(y=0.3, line_width=3, line_color='gray')
fig.update_yaxes(visible=False)
fig.update_xaxes(visible=False)
fig.update_layout(height=300, hovermode=False, showlegend=False,
                  title="Medicare Part D Prescription Claims in 2021", title_x=0.5, title_y=0.9,
                  xaxis_range=[-0.5,4.6], yaxis_range=[-0.2,2.2],                 
                  margin=dict(t=90, b=0, l=70, r=70),
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=35, color='#8a8d93'),
                  font=dict(size=16, color='#8a8d93')
                 )

In [ ]:
msno.bar(df_prs)

In [ ]:
df_prs['Prscrbr_Type'] = df_prs['Prscrbr_Type'].fillna('NA')
df_prs = df_prs.fillna(0)
df_prs[df_prs.columns[3:31]] = df_prs[df_prs.columns[3:31]].astype('int64')
df_prs.Bene_Avg_Risk_Scre = df_prs.Bene_Avg_Risk_Scre.round(2)
df_prs.head(3)

In [ ]:
msno.bar(df_prs)

## How Many Providers are Located in Your State in 2021?

In [ ]:
df_state = df_prs.groupby('Prscrbr_State_Abrvtn').agg(
    Prsc_cnt=pd.NamedAgg(column="PRSCRBR_NPI", aggfunc="nunique"),
    Tot_Day_Suply=pd.NamedAgg(column="Tot_Day_Suply", aggfunc="sum"),
    Tot_Clms = pd.NamedAgg(column="Tot_Clms", aggfunc="sum"),
    Tot_Drug_Cst=pd.NamedAgg(column="Tot_Drug_Cst", aggfunc="sum"),
    Tot_Benes=pd.NamedAgg(column="Tot_Benes", aggfunc="sum")
)

In [ ]:
for col in df_state.columns:
    df_state[col] = df_state[col].astype('str')
df_state = df_state.reset_index()
df_state.head(2)

In [ ]:
df_state['text'] = df_state['Prscrbr_State_Abrvtn'] + '<br>' + \
    'Prescribers' + ': ' + df_state['Prsc_cnt']  + '<br>' + \
    "Day’s Supply"+ ': ' + df_state['Tot_Day_Suply']  + '<br>' + \
    "Drug Cost"+ ': ' + df_state['Tot_Drug_Cst']   + '<br>' + \
    "Beneficiaries"+ ': ' + df_state['Tot_Benes']


fig = go.Figure(data=go.Choropleth(
    locations=df_state['Prscrbr_State_Abrvtn'], # Spatial coordinates
    z=df_state['Tot_Clms'].astype('int'),  # Data to be color-coded
    locationmode='USA-states', # set of locations match entries in `locations`
    colorscale='Reds',
    autocolorscale=False,
    text=df_state['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Total Claims"
))

fig.update_layout(
    title_text='How Many Providers are Located in Your State in 2021? <br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

## How are the Claims Paid?

In [ ]:
df_drugs = df_prs[[ 'Tot_Clms','Tot_Drug_Cst','Brnd_Tot_Drug_Cst','Gnrc_Tot_Drug_Cst','Othr_Tot_Drug_Cst',
                   'MAPD_Tot_Drug_Cst','PDP_Tot_Drug_Cst','LIS_Drug_Cst','NonLIS_Drug_Cst',
                   'Opioid_Tot_Clms','Opioid_Tot_Drug_Cst',
                   'Opioid_LA_Tot_Clms','Opioid_LA_Tot_Drug_Cst',
                   'Antbtc_Tot_Clms','Antbtc_Tot_Drug_Cst',
                   'Antpsyct_GE65_Tot_Clms','Antpsyct_GE65_Tot_Drug_Cst']].sum().round(0)
df_drugs

In [ ]:
drug_brand = [df_drugs['Brnd_Tot_Drug_Cst'],df_drugs['Gnrc_Tot_Drug_Cst'], df_drugs['Othr_Tot_Drug_Cst']]
drug_brand

In [ ]:
drug_flag = [df_drugs['Opioid_Tot_Drug_Cst'], df_drugs['Opioid_LA_Tot_Drug_Cst'],
     df_drugs['Antbtc_Tot_Drug_Cst'], df_drugs['Antpsyct_GE65_Tot_Drug_Cst']]
drug_flag

In [ ]:
percent = ((drug_flag[0]+drug_flag[1]+drug_flag[2])/df_drugs['Tot_Drug_Cst']).round(4) 
print('Flagged drug percent: {:.2%}'.format(percent))

In [ ]:
drug_plan = [df_drugs['MAPD_Tot_Drug_Cst'], df_drugs['PDP_Tot_Drug_Cst']]
drug_plan

In [ ]:
drug_Subsidy = [df_drugs['LIS_Drug_Cst'], df_drugs['NonLIS_Drug_Cst']]
drug_Subsidy

In [ ]:
fig = make_subplots(rows=1, cols=4, vertical_spacing=1, specs=[[{"type": "pie"},{"type": "pie"},{"type": "pie"},{"type": "pie"}]],
                    column_widths=[0.25, 0.25, 0.25, 0.25], 
                    subplot_titles=("Flagged Drug <br>", "Drug Brand <br>", "Insurance Plans <br>", "Covered By Subsidy"))

fig.add_trace(go.Pie(values=drug_flag, labels=['Opioid','Long Acting Opioid','Antibiotic', 'Antpsyct for Elderly'], 
                     marker_colors=['#334668','#4c78a8','#72b7b2', '#334668'],
                     hole=0.5, rotation=90, showlegend=False, 
                     hoverinfo='label+percent+value', textinfo='label'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=drug_brand, labels=['Brand Drugs','Generic Drugs','Other Drugs'],
                     marker_colors=['#334668','#4c78a8','#72b7b2'],
                     hole=0.5, rotation=90, showlegend=False, 
                     hoverinfo='label+percent+value', textinfo='label'), 
                     row=1, col=2)
fig.add_trace(go.Pie(values=drug_plan, labels=['MAPD Plans','PDP Plans'],
                     marker_colors=['#4c78a8','#72b7b2'],
                     hole=0.5, rotation=90, showlegend=False, 
                     hoverinfo='label+percent+value', textinfo='label'), 
                     row=1, col=3)
fig.add_trace(go.Pie(values=drug_Subsidy, labels=['Low-Income Subsidy','Non Low-Income Subsidy'], 
                     marker_colors=['#4c78a8','#72b7b2'],
                     hole=0.5, rotation=90, showlegend=False, 
                     hoverinfo='label+percent+value', textinfo='label'), 
                     row=1, col=4)
# styling
fig.update_traces(hovertemplate=None, marker=dict(line=dict(width=0)))
fig.update_layout(height=360, barmode='stack',
                  margin=dict(t=70, b=40, l=0, r=70), 
                  title="<span style='font-size:40px; font-family:Times New Roman'>How are the Claims Paid? </span>",
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  font=dict(size=13, color='#8a8d93'))
fig.show()

## What are the providers, claims, and costs in each specialty?

In [ ]:
df_prs2 = df_prs[['PRSCRBR_NPI','Tot_Clms','Tot_Drug_Cst','Prscrbr_Type']].drop(df_prs[df_prs['Prscrbr_Type']=='NA'].index)

In [ ]:
df_specialty = df_prs2.groupby('Prscrbr_Type').agg({'PRSCRBR_NPI':'count', 'Tot_Clms':'sum', 'Tot_Drug_Cst':'sum'}).reset_index()
df_specialty = df_specialty.sort_values(by=['PRSCRBR_NPI'], ascending=False)
df_specialty['Prscrbr_Type'] = df_specialty['Prscrbr_Type'].apply(lambda x: x.replace(" ","_"))
df_specialty.head(5)

In [ ]:
# convert to dict (method 1)
specialty_txt = dict(zip(df_specialty['Prscrbr_Type'].tolist(), df_specialty['PRSCRBR_NPI'].tolist()))
# convert to dict (method 2)
# data = df_specialty.set_index('Prscrbr_Type').to_dict()['Tot_Drug_Cst']

In [ ]:
wordcloud = WordCloud(background_color='white', width=800, height=400, max_words=50).generate_from_frequencies(specialty_txt)
# wordcloud = WordCloud(background_color='#333', width=800, height=400, max_words=50).generate_from_frequencies(specialty_txt)
plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.title('Which Specialty Type has the most providers? \n', fontsize=40, fontfamily='Times New Roman')
plt.axis("off")
plt.show() 

In [ ]:
df_specialty[['Tot_Clms','Tot_Drug_Cst']]= (df_specialty[['Tot_Clms','Tot_Drug_Cst']]/100000).astype('int64')

In [ ]:
df_specialty.head(2)

In [ ]:
df_claim_10 = df_specialty.sort_values(by=['Tot_Clms'], ascending=False).head(10)
df_claim_10

In [ ]:
df_cost_10 = df_specialty.sort_values(by=['Tot_Drug_Cst'], ascending=False).head(10)
df_cost_10

In [ ]:
# chart
fig = make_subplots(rows=2, cols=1, horizontal_spacing=0.01, vertical_spacing=0.04)                    
fig.add_trace(go.Bar(x=df_claim_10['Prscrbr_Type'], y=df_claim_10['Tot_Clms'], marker_color='#56627d', name='total claim', 
                     text = df_claim_10['Tot_Clms'].map('{:,}'.format), textposition = "inside",showlegend=True, hovertemplate='{x}'+ '{y}'
                     ), row=1, col=1)
fig.add_trace(go.Bar(x=df_cost_10['Prscrbr_Type'], y=df_cost_10['Tot_Drug_Cst'], marker_color='#72b7b2', name='total cost', 
                     text = df_cost_10['Tot_Drug_Cst'].map('{:,}'.format), textposition = "inside",showlegend=True, hovertemplate='{x}'+ '{y}'
                     ), row=2, col=1)


# styling
fig.update_xaxes(visible=True, categoryorder='total ascending', tickangle =15 ,row=1, col=1)
fig.update_xaxes(visible=True, categoryorder='total ascending', tickangle =15, row=2, col=1)
fig.update_yaxes(visible=False)
fig.update_traces(hovertemplate=None, marker=dict(line=dict(width=0)))
fig.update_layout(height=700, hovermode='x unified',
                  title="<span style='font-size:45px; font-family:Times New Roman'>Top 10 Claims & Costs (M) on Specialty </span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=9, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'),
                  legend=dict(title="", orientation="v", yanchor="bottom", xanchor="center", x=0.1, y=0.9,
                              bordercolor="#fff", borderwidth=0.5, font_size=13)
                  )

fig.show()

In [ ]:
# chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, horizontal_spacing=0.01, vertical_spacing=0.04)                    
fig.add_trace(go.Bar(x=df_claim_10['Prscrbr_Type'], y=df_claim_10['Tot_Clms'], marker_color='#56627d', name='total claim', 
                     text = df_claim_10['Tot_Clms'].map('{:,}'.format), textposition = "inside",showlegend=True,
                     hovertemplate = '%{y:,}'       
                     ), row=1, col=1)

fig.add_trace(go.Scatter(x=df_claim_10['Prscrbr_Type'], y=-df_claim_10['Tot_Drug_Cst'],
                         text=df_claim_10['Tot_Drug_Cst'], textposition='middle center',
                  textfont=dict(color='#E58606'), hovertemplate = '%{text:,}' , 
                  mode='lines+markers+text',
                  marker=dict(color=['#56627d','#c6ccd8','#334668','#56627d','#c6ccd8',
                       '#334668','#56627d','#56627d','#56627d','#334668'],
                              size=list(2**(np.log(df_claim_10['Tot_Clms']))/3)),
                  line=dict(color='#52BCA3', width=1, dash='dash'),
                  name='total cost')
                       , row=2, col=1)

# styling
fig.update_xaxes(visible=True, categoryorder='trace',  row=1, col=1)
fig.update_xaxes(showgrid=False, mirror="allticks", side='top',  row=2, col=1)
fig.update_yaxes(visible=False)
#fig.update_traces(hovertemplate=None, marker=dict(line=dict(width=0)))
fig.update_layout(height=700, 
                  title="<span style='font-size:40px; font-family:Times New Roman'>Top 10 Claims (Million) Specialty and Costs (Million)</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=9, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'),
                  legend=dict(title="", orientation="v", yanchor="bottom", xanchor="center", x=0.9, y=0.95,
                              bordercolor="#fff", borderwidth=0.5, font_size=13),
                  hovermode='x unified'
                  )

fig.show()

In [ ]:
#df_cc10 = df_specialty.loc[df_specialty['Prscrbr_Type'].isin(df_cost_10['Prscrbr_Type'])|
#                df_specialty['Prscrbr_Type'].isin(df_claim_10['Prscrbr_Type'])].sort_values(by=['Tot_Clms'], ascending=False)

## How are the Average Patient Health Condition?

In [ ]:
# Distribution of beneficiaries' age 
fig, axes = plt.subplots(1, 2, figsize=(18, 5))

sns.distplot(df_prs['Bene_Avg_Risk_Scre'], kde = True, color='green',
             kde_kws={"color": "b", "alpha": 0.8}, bins = 20, ax=axes[0])
axes[0].set_title("Beneficiaries Average Risk Score Distribution \n",fontsize = 14)
axes[0].set_xlabel("\n Beneficiary Average Risk Score", fontsize = 14)

sns.boxplot(x=df_prs['Bene_Avg_Risk_Scre'], palette="pastel", ax=axes[1])
axes[1].set_xlabel("\n Beneficiary Average Risk Score", fontsize = 14)
axes[1].set_title("Beneficiary Average Risk Score Boxplot \n", fontsize = 14)
plt.show()

In [ ]:
df_con = df_prs['Bene_Avg_Risk_Scre'].value_counts().rename_axis('Bene_Avg_Risk_Scre').reset_index(name='counts')
df_con.sort_values(by=['Bene_Avg_Risk_Scre'], ascending=True)

In [ ]:
df_prs["Risk_Scre_cat"] = pd.cut(df_prs['Bene_Avg_Risk_Scre'],
                               bins=[0, 0.5, 1, 1.5,  2, 3, 4, 5, np.inf],
                               labels=[1, 2, 3, 4, 5, 6,7,8])

In [ ]:
fig = px.histogram(df_prs, x="Risk_Scre_cat", histnorm='probability density',
                  title="How are the Average Patient Health Condition?",
            width=800, height=400, 
            labels={ # replaces default labels by column name
                "Risk_Scre_cat": "Beneficiary Average Risk Score"
            },
            template="plotly_dark",  # template="simple_white",
                   text_auto=True  
            )
fig.data[0].marker.line.width = 1
fig.data[0].marker.line.color = "white"
fig.update_layout(xaxis = dict(tickmode = 'array',
                               tickvals = [1, 2, 3, 4, 5, 6, 7, 8],
                               ticktext=["0-0.5", "0.5-1", "1-1.5", "1.5-2","2-3", "3-4", "4-5", ">5"]
                              ),
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=40, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'))
fig.show()

## How Prescibers Cluster on Flagged Drugs Providing?

I notice that opioids, long-acting opioids, antibiotic drugs, and Antpsyct drugs are specifically labeled and aggregated in the Medicare Pard D Presciber summary form. I realize that the crisis of abuse, waste, and fraud of drugs, especially the four labeled drugs mentioned above, is real. The issure should be take care of , whether from the perspective of preventing fraud and abuse of medical insurance funds of from the perspective of patients' health. However, identiying such a healthcare provider looks like finding a needle in a haystack. I will use clustering method to filter the providers and identify suspicious ones. This way, we reduce the size of the search pool, so that finding a needle becomes easy.   

In this section, I selected some features that I believe are important for identifying fraudulent behavior by prescribers on flagged drugs. These features are all related to the four flagged drugs. I found that the flagged drugs accounted for 2.82% of the total drug cost in the flagged drugs analysis selection. Including too much irrelevant information would interfere with the achieved cluster goals.

In [ ]:
# Correlation matrix
df_prepare = df_prs[['Tot_Clms','Opioid_Tot_Clms','Opioid_Tot_Drug_Cst','Opioid_Tot_Suply', 'Opioid_Tot_Benes', 'Opioid_Prscrbr_Rate', 
             'Opioid_LA_Tot_Clms', 'Opioid_LA_Tot_Drug_Cst', 'Opioid_LA_Tot_Suply', 'Opioid_LA_Tot_Benes', 'Opioid_LA_Prscrbr_Rate', 
             'Antbtc_Tot_Clms','Antbtc_Tot_Drug_Cst', 'Antbtc_Tot_Benes', 
             'GE65_Tot_Clms', 'Antpsyct_GE65_Tot_Clms', 'Antpsyct_GE65_Tot_Drug_Cst', 'Antpsyct_GE65_Tot_Benes']]
corr = df_prepare.corr()
fig = go.Figure(data=go.Heatmap(z=np.array(corr),x=corr.columns.tolist(),y=corr.columns.tolist(),xgap = 1,ygap = 1))
fig.update_layout(margin = dict(t=25,r=0,b=200,l=200),width = 1000, height = 700)
fig.show()

From the above correlation matrix, I found that Tot_clms have high positive correlation with Tot_suply and Tot_Benes for flagged drugs.  Some drug cost are high correlated with claims like  Opioid drugs and long acting Opioid drugs, but some not, like  Antibiotic drugs and Antipsychotics for the elderly. I will use mean cost and prescribe rate of flagged drugs as the key metrics.


I will remove those correlation features with Tot_clms for flagged drugs. Prscrbr_Rate features of Opioid drugs and long acting Opioid drugs are not high correlated with other features, so I will calculate Prscrbr_Rate for Antibiotic drugs and Antipsychotics

In [ ]:
df_prepare['Opioid_mean_cost'] = df_prepare['Opioid_Tot_Drug_Cst']/df_prepare['Opioid_Tot_Clms']
df_prepare['Opioid_LA_mean_cost'] = df_prepare['Opioid_LA_Tot_Drug_Cst']/df_prepare['Opioid_LA_Tot_Clms']
df_prepare['Antbtc_mean_cost'] = df_prepare['Antbtc_Tot_Drug_Cst']/df_prepare['Antbtc_Tot_Clms']
df_prepare['Antpsyct_mean_cost'] = df_prepare['Antpsyct_GE65_Tot_Drug_Cst']/df_prepare['Antpsyct_GE65_Tot_Clms']
df_prepare['Antbtc_Prscrbr_Rate'] = df_prepare['Antbtc_Tot_Clms']*100/df_prepare['Tot_Clms']
df_prepare['Antpsyct_Prscrbr_Rate'] = df_prepare['Antpsyct_GE65_Tot_Clms']*100/df_prepare['GE65_Tot_Clms']


In [ ]:
df_prepare.head(2)

In [ ]:
df_prepare = df_prepare.fillna(0)
df_clean = df_prepare[['Opioid_mean_cost','Opioid_LA_mean_cost','Antbtc_mean_cost','Antpsyct_mean_cost', 
                       'Opioid_Prscrbr_Rate', 'Opioid_LA_Prscrbr_Rate','Antbtc_Prscrbr_Rate',  'Antpsyct_Prscrbr_Rate']]

In [ ]:
df_clean.head(2)

In [ ]:
corr1 = df_clean.corr()
fig = go.Figure(data=go.Heatmap(z=np.array(corr1),x=corr1.columns.tolist(),y=corr1.columns.tolist(),xgap = 1,ygap = 1))
fig.update_layout(margin = dict(t=25,r=0,b=200,l=200),width = 1000, height = 700)
fig.show()

In [ ]:
#Scaling
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_clean)
df_scaled = pd.DataFrame(df_scaled, columns= df_clean.columns )
df_scaled.head(2)

In [ ]:
pca = PCA()
pca.fit(df_scaled)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

fig = px.line(x=np.arange(1,exp_var_cumul.shape[0]+1), y=exp_var_cumul, markers=True, labels={'x':'# of components', 'y':'Cumulative Explained Variance'})

#fig.add_shape(type='line', line=dict(dash='dash'),x0=0, x1=30, y0=0.95, y1=0.95)

fig.show()

I find that the cumulative variance growing stablely and doesn't form an obvious elbow This is because the features that I selected are not high correlated. I also find that the number of dimensions is 7 while preserving 95% of its variance. 

## Clustering

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
# Quick examination numbers of clusters by using elbow method.
print('Elbow Method to determine the number of clusters:')
plt.figure(figsize=(10,5))
Elbow_M = KElbowVisualizer(KMeans(), k=10) # metric default: "distortion", mean sum of squared distances to centers
# silhouette: mean ratio of intra-cluster and nearest-cluster distance
Elbow_M.fit(df_scaled)
Elbow_M.show()

In [ ]:
#KMeans Clustering

kmeans = KMeans(n_clusters=6,max_iter=50, random_state=42).fit(df_scaled)

In [ ]:
#centriods
centriods = kmeans.cluster_centers_
centriods = pd.DataFrame(centriods, columns= df_clean.columns)
centriods.index.name = 'Cluster'
centriods

In [ ]:
centriods_orignal = scaler.inverse_transform(centriods)
centriods_orignal = pd.DataFrame(centriods_orignal.round(2), columns= df_clean.columns)
centriods_orignal.index.name = 'Cluster'
centriods_orignal

In [ ]:
meancost_centriods = centriods_orignal.reset_index().melt(id_vars=['Cluster'], 
                                                          value_vars=['Opioid_mean_cost', 'Opioid_LA_mean_cost',
                                                                     'Antbtc_mean_cost', 'Antpsyct_mean_cost'],
                                                         var_name='Labeled Drugs', value_name='Mean Cost Cluster Centriods')

prsrate_centriods = centriods_orignal.reset_index().melt(id_vars=['Cluster'], 
                                                          value_vars=['Opioid_Prscrbr_Rate', 'Opioid_LA_Prscrbr_Rate',
                                                                     'Antbtc_Prscrbr_Rate', 'Antpsyct_Prscrbr_Rate'],
                                                        var_name='Labeled Drugs', value_name='Prscrbr Rate Cluster Centriods')

plt.style.use('default')  
fig, axes = plt.subplots(2, 1, figsize=(18, 12))

pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60", '#56627d','#c6ccc0']
sns.barplot(data=meancost_centriods, x="Labeled Drugs", y="Mean Cost Cluster Centriods", hue="Cluster",palette= pal, ax=axes[0])
axes[0].set_title("Cluster Centroids of  Mean Cost on the Claim for Labeled Drugs \n",fontsize = 14)

sns.barplot(data=prsrate_centriods, x="Labeled Drugs", y="Prscrbr Rate Cluster Centriods", hue="Cluster",palette= pal, ax=axes[1])
axes[1].set_title("Cluster Centroids of Prscrbr Rate for Labeled Drugs", fontsize = 14)
plt.show()


## Clustering Pattern Analysis

Since this is an unsupervised clustering. We do not have a tagged feature to evaluate or score our model. In this section, I study the clustering patterns and identify the characteristics of the labeled drug claims on each cluster. 

In [ ]:
df_clean["clusters"]= kmeans.labels_

In [ ]:
# Plotting countplot of clusters
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60", '#56627d','#c6ccc0']
ax = sns.countplot(x=df_clean["clusters"], palette= pal)
ax.set_title("Distribution Of The Clusters")
#ax.grid(True) 

total = len(df_clean["clusters"])
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width() / 2 -0.1 # To adjust the position of the percentage value
    y = p.get_y() + p.get_height() + 0 # To adjust the position of the percentage value
    ax.annotate(percentage, (x, y), size = 12)
plt.show()

In [ ]:
# Lets see the Key metrics of each clusters
for name in df_clean.columns[:-1]:
    sns.set(rc={"figure.figsize":(8, 4)})
    sns.boxplot(x="clusters",y=name, data=df_clean).set_title(name + " by Clusters")
    plt.show()

In [ ]:
for i in range(df_clean.shape[1]):
    print ("\n" + df_clean.columns[i] + "describled by clusters \n")
    for  j in range(0,6,1):
        print ("Cluster:", str(j))
        print(df_clean.iloc[:,i][df_clean['clusters']==j].describe())


### Cluster pattern: Which Clusters Need Attention?

For each flaged drugs claims, we have four patterns, Low rate and Low cost, High rate and Low cost, Low rate and High cost, 	High rate and High cost. If the cluster pattern shows low prscriber rate and low mean cost for flagged drugs, I consider the claims are normal except the outliers. If the pattern shows high prscriber rate and low mean cost, I will take a routinee check and see if there are specilty requirements for medical, and then check the outliers. If the cluster pattern shows high prscriber rate and high mean cost, or low prscriber rate but high mean cost, the claims are suspicious.  

In [ ]:
pattern = pd.DataFrame({'Cluster': [0,1,2,3,4,5], 'Percentage':[0.702,0.1630,0.0340,0.006,0.025,0.07],
                       'Pattern':['Low cost, Low rate for flagged drugs',
                                  'Low Antbtc_mean_cost, High Antbtc_Prscrbr_Rate',
                                  'High Opioid_mean_cost,  Low Opioid_Prscrbr_Rate, High Opioid_LA_mean_cost,High Opioid_LA_Prscrbr_Rate',
                                  'High Antbtc_mean_cost, Low Antbtc_Prscrbr_Rate',
                                  'High Antpsyct_mean_cost, High Antpsyct_Prscrbr_Rate',
                                  'Low  Opioid_mean_cost, High Opioid_Prscrbr_Rate'
                                ],
                       'Label':['Normal','Low Suspicious on Antbtc Claim','High Suspicious on Opioid Claim','Moderate Suspicious on Antbtc Claim','High Suspicious on  Antpsyct Claim', 'Low Suspicious on Opioid Claim']})
pattern 

In [ ]:
pattern_detail = pd.DataFrame({'Cluster': [0,1,2,3,4,5], 'Percentage':[0.702,0.1630,0.0340,0.006,0.025,0.07],
                        'Opioid_mean_cost: Q1-M-Q3':['0-0-4','0-0-0','42-62-99','0-0-0','0-0-0','4-6-9'],
                        'Opioid_Prscrbr_Rate (%): Q1-M-Q3':['0-0-0','0-0-0','3-7-25','0-0-0','0-0-0','30-39-52'],
                        'Opioid_LA_mean_cost: Q1-M-Q3':['0-0-0','0-0-0','102-213-362','0-0-0','0-0-0','0-0-0'],
                        'Opioid_LA_Prscrbr_Rate (%): Q1-M-Q3':['0-0-0','0-0-0','15-23-31','0-0-0','0-0-0','0-0-0'],
                        'Antbtc_mean_cost: Q1-M-Q3':['0-0-12','5-7-11','0-11-21','945-1248-1782','0-0-0','0-3-8'],
                        'Antbtc_Prscrbr_Rate (%): Q1-M-Q3':['0-0-0','41-54-72','0-2-3','3-5-8','0-0-0','0-4-23'],
                        'Antpsyct_mean_cost: Q1-M-Q3':['0-0-0', '0-0-0','0-0-23','0-0-0','68-206-472','0-0-0'],
                        'Antpsyct_Prscrbr_Rate (%): Q1-M-Q3':['0-0-0','0-0-0','0-0-1','0-0-0','15,21,29','0-0-0'],
                        'Pattern':['Low cost, Low rate for flagged drugs',
                                  'Low Antbtc_mean_cost, High Antbtc_Prscrbr_Rate',
                                  'High Opioid_mean_cost,  Low Opioid_Prscrbr_Rate, High Opioid_LA_mean_cost,High Opioid_LA_Prscrbr_Rate',
                                  'High Antbtc_mean_cost, Low Antbtc_Prscrbr_Rate',
                                  'High Antpsyct_mean_cost, High Antpsyct_Prscrbr_Rate',
                                  'Low  Opioid_mean_cost, High Opioid_Prscrbr_Rate',
                                ],
                       'Label':['Normal','Low Suspicious on Antbtc Claim','High Suspicious on Opioid Claim','Moderate Suspicious on Antbtc Claim','High Suspicious on  Antpsyct Claim', 'Low Suspicious on Opioid Claim']})
pattern_detail 

In [ ]:
df_clean['Prscrbr_Type']=df_prs['Prscrbr_Type']
df_clean['PRSCRBR_NPI']=df_prs['PRSCRBR_NPI']

### What Specialty Types are included in the suspicious cluster? How do each specialty type claims labeled drugs in suspicious cluster differ from it claims in other clusters?

#### Specialty Types in Cluster 2, Labled as High Suspicious on Opioid Claim

In [ ]:
Prscrbr_1 = df_clean.Prscrbr_Type[df_clean['clusters']==2].value_counts().rename_axis('Prscrbr_Type').reset_index(name='counts')
Prscrbr_1

#### Choose Family Practice from Specialty Types, Compare Opioid/LA-Opioid claims (mean cost and prscrbr rate) in cluster 2 with those in other clusters 

In [ ]:
# Compare Opioid_mean_cost by clusters.
fig = px.box(df_clean[df_clean['Prscrbr_Type']=='Family Practice'], x="clusters", y="Opioid_mean_cost", hover_data=["PRSCRBR_NPI"])
fig.update_layout(height=500,   
                  title="<span style='font-size:25px; font-family:Times New Roman'> Box Plots of Opioid Mean Claim Cost Offered by Family Practice Providers</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=15, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'),
                  xaxis=dict(#tickvals = [0, 1, 2, 3, 4, 5],ticktext=['Low Suspicious on Antbtc Claim','High Suspicious on Opioid Claim','Normal','Low Suspicious on Opioid Claim','Moderate Suspicious on Antbtc Claim','High Suspicious on  Antpsyct Claim'],
                             title="<span style='font-size:20px; font-family:Times New Roman'>Clusters</span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                  yaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Opioid Mean Cost </span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                )
fig.show()

In [ ]:
# Compare Opioid_Prscrbr_Rate by clusters.
fig = px.box(df_clean[df_clean['Prscrbr_Type']=='Family Practice'], x="clusters", y="Opioid_Prscrbr_Rate", hover_data=["PRSCRBR_NPI"])
fig.update_layout(height=500,   
                  title="<span style='font-size:25px; font-family:Times New Roman'> Box Plots of Opioid Prescription Rate of Family Practice Providers</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=15, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'),
                  xaxis=dict(#tickvals = [0, 1, 2, 3, 4, 5],ticktext=['Low Suspicious on Antbtc Claim','High Suspicious on Opioid Claim','Normal','Low Suspicious on Opioid Claim','Moderate Suspicious on Antbtc Claim','High Suspicious on  Antpsyct Claim'],
                             title="<span style='font-size:20px; font-family:Times New Roman'>Clusters</span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                  yaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Opioid Prescription Rate </span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                )
fig.show()

In [ ]:
# Compare Opioid_mean_cost by clusters.
fig = px.box(df_clean[df_clean['Prscrbr_Type']=='Family Practice'], x="clusters", y="Opioid_LA_mean_cost", hover_data=["PRSCRBR_NPI"])
fig.update_layout(height=500,   
                  title="<span style='font-size:25px; font-family:Times New Roman'> Box Plots of Long-acting Opioid Mean Claim Cost  Offered by Family Practice Providers</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=15, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'),
                  xaxis=dict(#tickvals = [0, 1, 2, 3, 4, 5],ticktext=['Low Suspicious on Antbtc Claim','High Suspicious on Opioid Claim','Normal','Low Suspicious on Opioid Claim','Moderate Suspicious on Antbtc Claim','High Suspicious on  Antpsyct Claim'],
                             title="<span style='font-size:20px; font-family:Times New Roman'>Clusters</span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                  yaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Long-acting Opioid Mean Cost </span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                )
fig.show()

In [ ]:
# Compare Opioid_mean_cost by clusters.
fig = px.box(df_clean[df_clean['Prscrbr_Type']=='Family Practice'], x="clusters", y="Opioid_LA_Prscrbr_Rate", hover_data=["PRSCRBR_NPI"])
fig.update_layout(height=500,   
                  title="<span style='font-size:25px; font-family:Times New Roman'> Box Plots of Long-acting Opioid Prescription Rate of Family Practice Providers</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=15, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=10,color='#8a8d93'),
                  xaxis=dict(#tickvals = [0, 1, 2, 3, 4, 5],ticktext=['Low Suspicious on Antbtc Claim','High Suspicious on Opioid Claim','Normal','Low Suspicious on Opioid Claim','Moderate Suspicious on Antbtc Claim','High Suspicious on  Antpsyct Claim'],
                             title="<span style='font-size:20px; font-family:Times New Roman'>Clusters</span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                  yaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Long-acting Opioid Prescription Rate </span>",
                             zerolinewidth=0.1,ticklen=1,gridwidth=0),
                )
fig.show()

From the above box-plots, we found that Claims in Clusters 2 are significant different from those in other clusters.

## Who are suspicious providers? 

### Example: Detect Who Offer Anomalous Opioid Claims from Interventional Pain Management Providers 

### Method 1: direct method using scatter plots

In [ ]:
df_1 = df_clean[df_clean['clusters']==2].groupby('Prscrbr_Type').agg({'Opioid_mean_cost':'median','Opioid_Prscrbr_Rate':'median',
                                                                      }).reset_index()
df_1.head(2)

In [ ]:
df_2 = pd.merge(Prscrbr_1, df_1, on='Prscrbr_Type')
df_2.head(2)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_2.Opioid_mean_cost,
    y=df_2.Opioid_Prscrbr_Rate,
    mode='markers',
    text=df_2.Prscrbr_Type,
    hovertemplate =
    '<b>%{text}</b><br>'+
    '<b>Opioid_mean_cost</b>: %{x:.2f}<br>'+
    '<b>Opioid_Prscrbr_Rate (%)</b>: %{y:.0f}<br>'+
    '<b>Count(log2):%{marker.size:.0f}</b>',
    marker=dict(
        size=np.log2(df_2.counts),
        line=dict(
            width=2
        ),
    )
))
fig.update_layout(height=500, 
                  title="<span style='font-size:32px; font-family:Times New Roman'> Scatter Plot of Prscriber Types in Cluster 2</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=15, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=15,color='#8a8d93'),
                  xaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Opioid_mean_cost (median)</span>",
                             zerolinewidth=1,ticklen=5,gridwidth=1),
                  yaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Opioid_Prscrbr_Rate (median)</span>",
                             zerolinewidth=1,ticklen=5,gridwidth=1),
                  )
fig.show()

In [ ]:
# Why choose Interventional Pain Management data to analysis? The median values of Opioid_mean_cost and Opioid_Prscrbr_Rate are high, 
# as well count numuber is large enough for Unsupervised Anomaly Detection in the next step.  
df_ipm = df_clean.loc[(df_clean['Prscrbr_Type']=='Interventional Pain Management') & (df_clean['clusters']==2)]

In [ ]:
Q1_omc= np.quantile(df_ipm.Opioid_mean_cost,0.25)
Q3_omc= np.quantile(df_ipm.Opioid_mean_cost,0.75)
IQR_omc= Q3_omc - Q1_omc   
Upper_omc = (Q3_omc + 1.5*IQR_omc)

Q1_opr= np.quantile(df_ipm.Opioid_Prscrbr_Rate,0.25)
Q3_opr= np.quantile(df_ipm.Opioid_Prscrbr_Rate,0.75)
IQR_opr= Q3_opr - Q1_opr   
Upper_opr = (Q3_opr + 1.5*IQR_opr)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ipm.Opioid_mean_cost,
    y=df_ipm.Opioid_Prscrbr_Rate,
    mode='markers',
    text=df_ipm.PRSCRBR_NPI,
    hovertemplate =
    '<b>PRSCRBR_NPI</b>: %{text}<br>'+
    '<b>Opioid_mean_cost</b>: %{x:.2f}<br>'+
    '<b>Opioid_Prscrbr_Rate (%)</b>: %{y:.0f}<br>',
    marker=dict(
        size=7,
        line=dict(
            width=2
        ),
    )
))
fig.add_vrect(x0=Upper_omc, x1=df_ipm.Opioid_mean_cost.max(), line_width=0, fillcolor="#c6ccd8", opacity=0.2,
             annotation_text="Suspicious Points Region (Upper Boundary to Max)", annotation_position="top left")
fig.add_hline(y=Upper_opr, line_width=3, line_dash="dash", line_color="red",
             annotation_text="Opioid_Prscrbr_Rate Upper Boundary Line", 
             annotation_position="bottom right")
fig.update_layout(height=500, 
                  title="<span style='font-size:25px; font-family:Times New Roman'> Scatter Plot of Opioid Claims by Interventional Pain Management Providers in Cluster 2</span>",
                  margin=dict(t=130, b=10, l=15, r=15),               
                  plot_bgcolor='#333', paper_bgcolor='#333',
                  title_font=dict(size=15, color='#8a8d93', family="Times New Roman"),
                  font=dict(size=15,color='#8a8d93'),
                  xaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Opioid_mean_cost</span>",
                             zerolinewidth=1,ticklen=5,gridwidth=1),
                  yaxis=dict(title="<span style='font-size:20px; font-family:Times New Roman'>Opioid_Prscrbr_Rate</span>",
                             zerolinewidth=1,ticklen=5,gridwidth=1),
                  )
fig.show()



### Interventional Pain Management Providers Offering anomaly Opioids.

In [ ]:
Opioid_imp_0 = df_ipm.loc[(df_ipm['Opioid_mean_cost']>=Upper_omc)|(df_ipm['Opioid_Prscrbr_Rate']>=Upper_opr)]
print('Providers (NPI) in Interventional Pain Management who probably perform abnormal Opioid drugs on the claim:\n', 
      list(Opioid_imp_0.PRSCRBR_NPI))                                                                        

In [ ]:
Opioid_imp_0 = df.merge(Opioid_imp_0, how='inner',left_on='PRSCRBR_NPI', right_on='PRSCRBR_NPI')
print(Opioid_imp_0.shape)
Opioid_imp_0.head(2)

### Method 2: Unsupervised Anomaly Detection

In [ ]:
minmax = MinMaxScaler(feature_range=(0, 1))
df_ipm[['Opioid_mean_cost_scaled','Opioid_Prscrbr_Rate_scaled']] = minmax.fit_transform(df_ipm[['Opioid_mean_cost','Opioid_Prscrbr_Rate']])
X = df_ipm[['Opioid_mean_cost_scaled','Opioid_Prscrbr_Rate_scaled']].values

In [ ]:
#X1 = df_ipm['Opioid_mean_cost_scaled'].values.reshape(-1,1)
#X2 = df_ipm['Opioid_Prscrbr_Rate_scaled'].values.reshape(-1,1)

# X = np.concatenate((X1,X2),axis=1)

In [ ]:
def Anomaly_Detection_Plot(y_pred, df1, Z, threshold, name):
    n_inliers = len(y_pred) - np.count_nonzero(y_pred)
    n_outliers = np.count_nonzero(y_pred == 1)
    
    #df1 = df_ipm.copy()
    df1['outlier'] = y_pred.tolist()
    
    # sales - inlier feature 1,  profit - inlier feature 2
    inliers_omc= np.array(df1['Opioid_mean_cost_scaled'][df1['outlier'] == 0]).reshape(-1,1)
    inliers_opr = np.array(df1['Opioid_Prscrbr_Rate_scaled'][df1['outlier'] == 0]).reshape(-1,1)
    
    # sales - outlier feature 1, profit - outlier feature 2
    outliers_omc = df1['Opioid_mean_cost_scaled'][df1['outlier'] == 1].values.reshape(-1,1)
    outliers_opr = df1['Opioid_Prscrbr_Rate_scaled'][df1['outlier'] == 1].values.reshape(-1,1)
    print('OUTLIERS:',n_outliers,'INLIERS:',n_inliers)
    

    plt.figure(figsize=(8, 8))
    # fill blue map colormap from minimum anomaly score to threshold value
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),cmap=plt.cm.Blues_r)
    
    # draw red contour line where anomaly score is equal to thresold
    a = plt.contour(xx, yy, Z, levels=[threshold],linewidths=2, colors='red')
        
    # fill orange contour lines where range of anomaly score is from threshold to maximum anomaly score
    plt.contourf(xx, yy, Z, levels=[threshold, Z.max()],colors='orange')
    
    b = plt.scatter(inliers_omc, inliers_opr, c='white',s=20, edgecolor='k')
    
    c = plt.scatter(outliers_omc, outliers_opr, c='black',s=20, edgecolor='k')
       
    plt.axis('tight')   
    plt.legend([a.collections[0], b,c], ['learned decision function', 'inliers','outliers'],
           loc='lower right',  fontsize=15)
    plt.xlim((0, 1))
    plt.ylim((0, 1))
    plt.xlabel('Opioid_mean_cost_scaled')
    plt.ylabel('Opioid_Prscrbr_Rate_scaled')
    plt.title(name, fontsize=20, fontfamily='Times New Roman')
    plt.show()

In [ ]:
outliers_fraction = 0.075
xx , yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))

### K Nearest Neighbors (KNN)

In [ ]:
clf_KNN = KNN(contamination=outliers_fraction)
clf_KNN.fit(X)
# predict raw anomaly score
scores_pred_KNN = clf_KNN.decision_function(X) * -1
    
# prediction of a datapoint category outlier or inlier
y_pred_KNN = clf_KNN.predict(X)

# decision function calculates the raw anomaly score for every point
Z_clf_KNN = clf_KNN.decision_function(np.c_[xx.ravel(), yy.ravel()]) * -1
Z_clf_KNN = Z_clf_KNN.reshape(xx.shape)

# Use threshold value to consider a datapoint inlier or outlier
# threshold = stats.scoreatpercentile(scores_pred,100 * outliers_fraction)
threshold_KNN = percentile(scores_pred_KNN, 100 * outliers_fraction)

KNN_plot = Anomaly_Detection_Plot(y_pred_KNN, df_ipm.copy(), Z_clf_KNN,threshold_KNN, 'K Nearest Neighbors (KNN)')

### Histogram-base Outlier Detection (HBOS)

In [ ]:
clf_HBOS = HBOS(contamination=outliers_fraction)
clf_HBOS.fit(X)
# predict raw anomaly score
scores_pred_HBOS = clf_HBOS.decision_function(X) * -1
    
# prediction of a datapoint category outlier or inlier
y_pred_HBOS = clf_HBOS.predict(X)

# decision function calculates the raw anomaly score for every point
Z_clf_HBOS = clf_HBOS.decision_function(np.c_[xx.ravel(), yy.ravel()]) * -1
Z_clf_HBOS = Z_clf_HBOS.reshape(xx.shape)

# Use threshold value to consider a datapoint inlier or outlier
# threshold = stats.scoreatpercentile(scores_pred,100 * outliers_fraction)
threshold_HBOS = percentile(scores_pred_HBOS, 100 * outliers_fraction)

HBOS_plot = Anomaly_Detection_Plot(y_pred_HBOS, df_ipm.copy(), Z_clf_HBOS,threshold_HBOS, 
                                   'Histogram-base Outlier Detection (HBOS)')

### Isolation Forest

In [ ]:
clf_IForest = IForest(contamination=outliers_fraction,random_state=0)
clf_IForest.fit(X)
# predict raw anomaly score
scores_pred_IForest = clf_IForest.decision_function(X) * -1
    
# prediction of a datapoint category outlier or inlier
y_pred_IForest = clf_IForest.predict(X)

# decision function calculates the raw anomaly score for every point
Z_clf_IForest = clf_IForest.decision_function(np.c_[xx.ravel(), yy.ravel()]) * -1
Z_clf_IForest = Z_clf_IForest.reshape(xx.shape)

# Use threshold value to consider a datapoint inlier or outlier
# threshold = stats.scoreatpercentile(scores_pred,100 * outliers_fraction)
threshold_IForest = percentile(scores_pred_IForest, 100 * outliers_fraction)

IForest_plot = Anomaly_Detection_Plot(y_pred_IForest, df_ipm.copy(), Z_clf_IForest,threshold_IForest, 'Isolation Forest')

### Cluster-based Local Outlier Factor (CBLOF)

In [ ]:
clf_CBLOF = CBLOF(contamination=outliers_fraction,check_estimator=False, random_state=0)
clf_CBLOF.fit(X)
# predict raw anomaly score
scores_pred_CBLOF = clf_CBLOF.decision_function(X) * -1
    
# prediction of a datapoint category outlier or inlier
y_pred_CBLOF = clf_CBLOF.predict(X)

# decision function calculates the raw anomaly score for every point
Z_clf_CBLOF = clf_CBLOF.decision_function(np.c_[xx.ravel(), yy.ravel()]) * -1
Z_clf_CBLOF = Z_clf_CBLOF.reshape(xx.shape)

# Use threshold value to consider a datapoint inlier or outlier
# threshold = stats.scoreatpercentile(scores_pred,100 * outliers_fraction)
threshold_CBLOF = percentile(scores_pred_CBLOF, 100 * outliers_fraction)

CBLOF_plot = Anomaly_Detection_Plot(y_pred_CBLOF, df_ipm.copy(), Z_clf_CBLOF,threshold_CBLOF, 'Cluster-based Local Outlier Factor (CBLOF)')

### Interventional Pain Management Providers Offering anomaly Opioids.

In [ ]:
df_ipm['outlier'] = y_pred_IForest.tolist()

Opioid_imp_1 = df_ipm.loc[df_ipm['outlier'] == 1]

In [ ]:
print('List Providers (NPI) in Interventional Pain Management who offered anomaly opioid drugs on the claim:\n', 
      list(Opioid_imp_1.PRSCRBR_NPI))

In [ ]:
Opioid_imp = df.merge(Opioid_imp_1, how='inner',left_on='PRSCRBR_NPI', right_on='PRSCRBR_NPI')
print(Opioid_imp.shape)
Opioid_imp.head(2)